In [112]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot


import h5py
import pandas as pd

In [113]:
f=h5py.File(r'C:/Users/huboz/Desktop/Pjatk/HARDataset/kk_sword_basic.h5', 'r')

In [114]:

print(list(f.keys()))

['x', 'y_features', 'y_move', 'y_person']


In [115]:

x = f['x']
y = f['y_move']

In [116]:
print('(samples, timesteps, features) (samples,label) -> ', x.shape, y.shape)

(samples, timesteps, features) (samples,label) ->  (3642, 425, 64) (3642, 1)


In [117]:
x.shape, y.shape

((3642, 425, 64), (3642, 1))

x - 2549 sample
y=2550:3642

In [118]:
def trenujX():
    x1=[]
    for i in range(0, 2549):
        x1.append(x[i][63][424])

    return x1

In [119]:
def trenujY():
    y1=[]
    for i in range(0, 2549):
        y1.append(y[i][0])
    
    return y1

In [120]:
def testX():
    x2=[]
    for i in range(2550,3642):
        x2.append(x[i][63][424])

    return x2

In [121]:
def testY():
    y2=[]
    for i in range(2550,3642):
        y2.append(y[i][0])
    
    return y2

In [122]:
def load_group(filenames):
    loaded = list()
    for name in filenames:
        data = f
        loaded.append(data)
    # grupę stosów, tak aby funkcje były trzecim wymiarem
    loaded = dstack(loaded)
    return loaded

In [131]:
import numpy as np

X = x[0:3642]
Y = y[0:3642]

def Shuffle_Arr(c):
    np.random.shuffle(c)
    c = list(zip(X, Y))
    np.random.shuffle(c)
    
    return c

In [133]:
Shuffle_Arr(c)

[(array([[-4.37624847e+02, -1.87870941e+02,  1.57384167e+03, ...,
           3.05822777e-02,  4.00046931e-01, -3.65076575e-03],
         [-4.37598768e+02, -1.86647847e+02,  1.57359255e+03, ...,
           5.15600378e-02,  5.05041137e-01,  3.42427200e-02],
         [-4.37628131e+02, -1.85373899e+02,  1.57332979e+03, ...,
           7.14950794e-02,  6.04189629e-01,  7.60867942e-02],
         ...,
         [-4.88143002e+02, -1.23770636e+02,  1.57515158e+03, ...,
           3.42440599e-02,  4.23133321e-02,  6.35050782e-01],
         [-4.90778825e+02, -1.23222246e+02,  1.57499027e+03, ...,
           3.42396104e-02,  4.23796078e-02,  6.35048360e-01],
         [-4.93425323e+02, -1.22706917e+02,  1.57482141e+03, ...,
           3.42382797e-02,  4.24040794e-02,  6.35042565e-01]]),
  array([0], dtype=int64)),
 (array([[-5.87339172e+02, -5.97195663e+01,  1.46221399e+03, ...,
           2.88827080e-01,  1.43678445e-02,  6.46145891e-02],
         [-5.87724571e+02, -5.89135257e+01,  1.46235241e+03,

In [124]:
def load_dataset():
    
    Shuffle_Arr(c)
    
    TrainX = X[0:3642]
    TestX = X[2550:3642]
    TrainY = Y[0:3642]
    TestY = Y[2550:3642]

    TrainY = to_categorical(TrainY)
    TestY = to_categorical(TestY)
    #print(trenujX.shape, trenujY.shape, testX.shape, testY.shape)
    return TrainX, TrainY, TestX, TestY

In [125]:
TrainX, TrainY, TestX, TestY = load_dataset()

In [126]:
def evaluate_model(TrainX, TrainY, TestX, TestY):
    verbose, epochs, batch_size = 0, 10, 64 # 15, 64 domyslnie, dla 10, 64 działa najlepiej
    n_timesteps, n_features, n_outputs = TrainX.shape[1], TrainX.shape[2], TrainY.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.1)) # 0.5
    model.add(Flatten()) #dodalam
    model.add(Dropout(0.5)) #dodalam
    model.add(Flatten()) #dodalam
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(TrainX, TrainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(TestX, TestY, batch_size=batch_size, verbose=0)
    return accuracy

In [127]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


In [128]:
evaluate_model(TrainX, TrainY, TestX, TestY)

0.17124542593955994

In [129]:
def run_experiment(repeats=10):
    # load data
    TrainX, TrainY, testX, TestY = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(TrainX, TrainY, TestX, TestY)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)


In [130]:
run_experiment(repeats=10)

>#1: 17.125
>#2: 17.125
>#3: 17.125
>#4: 17.125
>#5: 17.125
>#6: 17.125
>#7: 17.125
>#8: 17.125
>#9: 17.125


KeyboardInterrupt: 